In [1]:
import fsl

In [2]:
import ants

In [3]:
import os
os.environ["TMPDIR"] = "/workspace/temp"


In [4]:
import nibabel as nib
from deepbrain import Extractor
import os
import pandas as pd
import numpy as np
from pathlib import Path
import time

In [ ]:
patients = pd.read_csv("idaSearch_7_07_2021 (3).csv")

def strip(file, output, name):
    img = nib.load(file).get_fdata()
    affine = nib.load(file).affine
    now = time.time()
    ext = Extractor()
    prob = ext.run(img) 
    mask = prob > 0.5
    stripped = (1 * mask).astype(np.uint8)
    stripped = nib.Nifti1Image(stripped, affine)
  
    brain = img[:]
    brain[~mask] = 0
    brain = nib.Nifti1Image(brain, affine)
    nib.save(brain, str(name + ".nii.gz"))
    #nib.save(stripped, str(name + "_mask.nii.gz"))
    print("Extraction time: {0:.2f} secs.".format(time.time() - now))
    
PET = patients.loc[(patients["Modality"] == "PET")]["Image ID"].values

for root, directories, files in os.walk("./data"):
    for file in files:
        if file.endswith("brain.nii.gz"):
            pass
        #elif Path(root+"brain.nii.gz").exists():
        #    pass
        elif int(file.split("_I")[-1][:-4]) in PET:
           # print(int(file.split("_I")[-1][:-4]))
            strip(os.path.join(root, file), root, file.split("_I")[-1][:-4])

In [5]:
from fsl.wrappers.flirt import flirt
def register(image_id, size="1"):
    brain = ants.image_read("brains/" + image_id + ".nii.gz")
    #mask = ants.image_read("masks/" + image_id+"_mask.nii.gz")
    ref = ants.image_read("MNI152_T1_1mm_brain.nii.gz")
    reg = "reg" if size == 1 else f"reg{size}"
    out = Path("registered/"+image_id + f"brain_{reg}.nii.gz")
    mat = Path("registered/"+image_id + f"brain_{reg}.mat")
    full = Path("registered/"+image_id + f"full_{reg}.nii.gz")
    mask_out = Path("registered/"+image_id + f"brain_mask_{reg}.nii.gz")
                   
    if not mat.exists():
        now = time.time()
        registered = ants.registration(ref, brain, type_of_transform = "SyN")
        output = ants.apply_transforms(ref, brain, registered['fwdtransforms'])
        print("Registration time: {0:.2f} secs.".format(time.time() - now))
        nib.save(ants.utils.convert_nibabel.to_nibabel(output), "registered/" + image_id + "_registered.nii.gz")#     if not full.exists():
#         now = time.time() 
#         flirt(src,ref,out=full,applyxfm=True,init=mat,v=True)
#         print("Full head time: {0:.2f} secs.".format(time.time() - now))
        
#     if not mask_out.exists():
#         now = time.time()
#         flirt(mask.as_posix(),ref,out=mask_out,applyxfm=True,init=mat,v=True)
#         print("Mask time: {0:.2f} secs.".format(time.time() - now))

In [26]:
alz = pd.read_csv("alzheimers.csv").drop_duplicates()
conv = pd.read_csv("converted.csv").drop_duplicates()
non = pd.read_csv("nonconverted.csv").drop_duplicates()
cont = pd.read_csv("control.csv").drop_duplicates()

In [27]:
for ids in alz["MRI_ID"]:
    if Path("registered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in cont["MRI_ID"]:
    if Path("registered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in conv["MRI_ID"]:
    if Path("registered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in non["MRI_ID"]:
    if Path("registered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

alz["MRI_ID"].size + cont["MRI_ID"].size + conv["MRI_ID"].size + non["MRI_ID"].size

In [37]:
from fsl.wrappers.flirt import flirt
def register(image_id, size="1"):
    brain = ants.image_read("petbrains/" + image_id + ".nii.gz")
    #mask = ants.image_read("masks/" + image_id+"_mask.nii.gz")
    ref = ants.image_read("MNI152_T1_1mm_brain.nii.gz")
    reg = "reg" if size == 1 else f"reg{size}"
    out = Path("petregistered/"+image_id + f"brain_{reg}.nii.gz")
    mat = Path("petregistered/"+image_id + f"brain_{reg}.mat")
    full = Path("petregistered/"+image_id + f"full_{reg}.nii.gz")
    mask_out = Path("petregistered/"+image_id + f"brain_mask_{reg}.nii.gz")
                   
    if not mat.exists():
        now = time.time()
        registered = ants.registration(ref, brain, type_of_transform = "SyN")
        output = ants.apply_transforms(ref, brain, registered['fwdtransforms'])
        print("Registration time: {0:.2f} secs.".format(time.time() - now))
        nib.save(ants.utils.convert_nibabel.to_nibabel(output), "petregistered/" + image_id + "_registered.nii.gz")#     if not full.exists():
#         now = time.time() 
#         flirt(src,ref,out=full,applyxfm=True,init=mat,v=True)
#         print("Full head time: {0:.2f} secs.".format(time.time() - now))
        
#     if not mask_out.exists():
#         now = time.time()
#         flirt(mask.as_posix(),ref,out=mask_out,applyxfm=True,init=mat,v=True)
#         print("Mask time: {0:.2f} secs.".format(time.time() - now))

for ids in alz["PET_ID"]:
    if Path("petregistered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in cont["PET_ID"]:
    if Path("petregistered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in conv["PET_ID"]:
    if Path("petregistered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))

for ids in non["PET_ID"]:
    if Path("petregistered/"+ str(ids) + "_registered.nii.gz").exists():
        pass
    else:
        try:
            register(str(ids))
        except:
            print("something went wrong with " + str(ids))


something went wrong with 26491
something went wrong with 282605
something went wrong with 242607
something went wrong with 248342
something went wrong with 27906
something went wrong with 287394
something went wrong with 274773
something went wrong with 26416
something went wrong with 26495


In [36]:
register("26487")

Registration time: 60.59 secs.


In [38]:
for ids in conv["PET_ID"]:
    print(ids)

257219
262695
264141
289809
311755
26248
26256
40888
270590
273887
290271
25349
26416
26487
26495
26503
30985
269760
315199
26670
36819
27006
27014
27018
44538
269486
250199
304706
246831
255912
265133
306200
301470
337879
266074
40892
280211
325139
257594
260445
286569
300121
339791
30494
50636
313240
313248
335295
345458
343408
312964
240860
258854
310820
257236
300390
300398
334281
29192
36202
36869
44571
286450
306224
312563
309766
346769
232196
247941
27805
27817
70935
74269
73283
245562
253121
307038
337891
332306
332312
356270
326241
340444
240029
251269
285728
33210
321665
56758
256393
298493
255187
257874
282465
285086
301924
307678
129531
73792
254131
260997
340991
275741
262731
274765
294559
273911
306411
266968
266972
300780
312063
336297
285753


In [ ]:
Path("MNI152_T1_1mm_brain.nii.gz").as_posix()

In [ ]:
root='./data/new_converts/ADNI'
convlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]


In [ ]:
root='./data/new_nonconverts/ADNI'
nonconvlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]


In [ ]:
len(set(convlist) & set(nonconvlist))

In [ ]:
root='./data/new_alzheimers/ADNI'
alzlist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]


In [ ]:
root='./data/new_control/ADNI'
controllist = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]


In [ ]:
set(controllist) & set(convlist)

In [ ]:
registered